In [3]:
import hashlib
import random
# ======================
# 1. INITIALIZE PARAMETERS
# ======================
p = 23  # Prime modulus
g = 5   # Generator
x = 6   # Secret key
y = pow(g, x, p)  # Public key
print("=== Schnorr ZKP Parameters ===")
print(f"Prime p: {p}")
print(f"Generator g: {g}")
print(f"Secret x: {x}")
print(f"Public key y: {y}")
# ======================
# 2. HONEST INTERACTIVE PROTOCOL (FIXED)
# ======================
def honest_schnorr_interactive():
    print("\n--- Honest Interactive Run ---")
    # Prover: choose random r, compute commitment t = g^r mod p
    r = 4
    t = pow(g, r, p)
    print(f"Prover commitment t = {t}")
    # Verifier: send random challenge e
    e = 1
    print(f"Verifier challenge e = {e}")
    # Prover: compute response s = r + e*x mod (p-1)
    s = (r + e * x) % (p-1)
    print(f"Response s = {s}")
    # Verifier: check if g^s ≡ t * y^e mod p
    verification = pow(g, s, p) == (t * pow(y, e, p)) % p
    print(f"Verification: {'Passed' if verification else 'Failed'}")
    return verification
# ======================
# 3. CHEATING PROVER
# ======================
def cheating_schnorr_interactive():
    print("\n--- Cheating Attempt ---")
    # Cheating prover doesn't know x
    fake_t = 9
    print(f"Prover fakes t = {fake_t}")
    # Verifier sends challenge
    e = 0
    print(f"Verifier challenge e = {e}")
    # Cheating prover strategies:
    if e == 0:
        # If challenge is 0, they can succeed by sending any random s
        s = random.randint(1, p-2)
        verification = pow(g, s, p) == fake_t
    else:
        # If challenge is 1, they cannot succeed without knowing x
        s = random.randint(1, p-2)
        verification = False  # Always fail when e=1 without knowing x

    print(f"Verification: {'Passed' if verification else 'Failed'}")
    return verification
# ======================
# 4. CHEATING PROBABILITY EXPERIMENT
# ======================
def cheating_probability_experiment(num_trials=100):
    print(f"\n--- Cheating Probability Experiment ---")
    success_count = 0
    for i in range(num_trials):
        # Cheating prover guesses commitment
        fake_t = random.randint(1, p-1)
        # Random challenge (0 or 1)
        e = random.randint(0, 1)
        # Cheater succeeds only if e=0
        if e == 0:
            # They can compute valid s for any fake_t when e=0
            s = random.randint(1, p-2)
            if pow(g, s, p) == fake_t:
                success_count += 1
    success_rate = success_count / num_trials
    print(f"Cheating success rate = {success_rate:.2f} (after {num_trials} runs)")
    return success_rate
# ======================
# 5. NON-INTERACTIVE SCHNORR (Fiat-Shamir)
# ======================
def schnorr_non_interactive(message="Hello Verifier"):
    print(f"\n--- Fiat–Shamir (Non-Interactive) ---")
    # Prover: choose random r
    r = 4
    t = pow(g, r, p)
    # Generate challenge using hash (Fiat-Shamir transform)
    hash_input = f"{t}{message}".encode()
    e = int(hashlib.sha256(hash_input).hexdigest(), 16) % 2
    print(f"Hash-based challenge = {e}")
    # Prover computes response
    s = (r + e * x) % (p-1)
    # Verifier recomputes challenge and verifies
    hash_input_verify = f"{t}{message}".encode()
    e_verify = int(hashlib.sha256(hash_input_verify).hexdigest(), 16) % 2
    verification = pow(g, s, p) == (t * pow(y, e_verify, p)) % p
    print(f"Verification: {'Passed' if verification else 'Failed'}")
    return verification
# ======================
# 6. RUN ALL COMPONENTS
# ======================
def main():
    print("=" * 50)
    print("SCHNORR ZERO-KNOWLEDGE PROOF DEMONSTRATION")
    print("=" * 50)
    # 1. Honest execution
    honest_schnorr_interactive()
    # 2. Cheating attempt
    cheating_schnorr_interactive()
    # 3. Non-interactive proof
    schnorr_non_interactive()
    # 4. Cheating probability experiment
    cheating_probability_experiment(100)
    print("\n" + "=" * 50)
    print("DEMONSTRATION COMPLETED SUCCESSFULLY!")
    print("=" * 50)
# Run the main function
if __name__ == "__main__":
    main()

=== Schnorr ZKP Parameters ===
Prime p: 23
Generator g: 5
Secret x: 6
Public key y: 8
SCHNORR ZERO-KNOWLEDGE PROOF DEMONSTRATION

--- Honest Interactive Run ---
Prover commitment t = 4
Verifier challenge e = 1
Response s = 10
Verification: Passed

--- Cheating Attempt ---
Prover fakes t = 9
Verifier challenge e = 0
Verification: Failed

--- Fiat–Shamir (Non-Interactive) ---
Hash-based challenge = 0
Verification: Passed

--- Cheating Probability Experiment ---
Cheating success rate = 0.05 (after 100 runs)

DEMONSTRATION COMPLETED SUCCESSFULLY!
